### Import Packages

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import the AI for Earth modules 
import sys
sys.path.append('C:\\wri_work\\ai4earth_modules')
import gppd_ai4earth.gppd_gen as gen

C:\Users\terry.yin\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
C:\Users\terry.yin\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:18: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


### Load Global Power Plant Database

In [3]:
GPPD_PATH = 'global_power_plant_database(jrc morocco chile AUS_year updated).csv'
OUTPUT_PATH = 'gppd_updated_with_generation_estimations.csv'
PLATTS_PATH = 'PLATTS MARCH 2017.csv'

In [4]:
# Load the global power plant database
gppd = pd.read_csv(GPPD_PATH)

### Initialize The Estimator and Create Parameter Mappings

In [5]:
# initialize an estimator
estimator = gen.model_runner.Estimator()

In [57]:
# Make parameter name mapping
param_name_map = {'primary_fuel':'fuel_type',
                  'capacity_mw':'capacity_mw',
                  'commissioning_year':'commissioning_year',
                  'country_long':'country',
                  'latitude':'lat',
                  'longitude':'lon',
                  'UTYPE':'turbine_type'}

# Needed parameters for each fuel type
model_params_map = {'Wind':
              {'fuel_type',
              'capacity_mw',
#               'estimating_year',
              'commissioning_year',
              'country',
              'lat',
              'lon'},
              
              'Solar':
              {'fuel_type',
              'capacity_mw',
#               'estimating_year',
              'commissioning_year',
              'country',
              'lat',
              'lon'},
              
              'Hydro':
              {'fuel_type',
              'capacity_mw',
#               'estimating_year'
              'country',
              'lat',
              'lon'},
              
              'Gas':
              {'fuel_type',
              'capacity_mw',
#               'estimating_year',
              'commissioning_year',
              'turbine_type',
              'country'}
             }

param2column = {v:k for k, v in param_name_map.items()}

In [58]:
# A solar sample
param_dict = {'fuel_type':'Solar',
              'capacity_mw':25,
              'estimating_year':2017,
              'commissioning_year':2013,
              'country':'United States of America',
              'lat':45,
              'lon':-110}
estimator.estimates(**param_dict)

NameError: name 'estimator' is not defined

### Impute Turbine Types for Gas Plants

In [7]:
# select gas plants from GPPD
gppd_gas = gppd[(gppd['primary_fuel'].isin(['Gas','Fossil Gas'])) &  
                        (gppd['gppd_idnr'].notnull()) & 
                        (gppd['wepp_id'].notnull())]
gppd_gas = gppd_gas[gppd_gas['wepp_id'].str.len() <= 7]

In [8]:
# Loading Platts data
platts = pd.read_csv(PLATTS_PATH,encoding='ISO-8859-1')
platts = platts[(platts['STATUS']=='OPR')] #& (platts['COUNTRY']!='USA')]
platts_gas_related_ids = platts.loc[platts['FUEL']=='GAS','LOCATIONID'].unique()
platts_gas_related = platts[platts['LOCATIONID'].isin(platts_gas_related_ids)]

In [9]:
# Extract Gas dominated plants from all plants
platts_gas_related['FUEL'].replace({'WSTH':'GAS'},inplace = True)

temp = platts_gas_related.groupby(['LOCATIONID','FUEL']).agg({'MW':'sum'})
platts_gas_fuel_proportion = temp.groupby(level = 0).apply(lambda x:x/float(x.sum()))
platts_gas_fuel_proportion.reset_index(inplace=True)

platts_pure_gas_ids = platts_gas_fuel_proportion.loc[platts_gas_fuel_proportion['MW'] >= 0.95,'LOCATIONID']
platts_pure_gas = platts[platts['LOCATIONID'].isin(platts_pure_gas_ids)]

In [10]:
# Extract gas plants operated under a single generating technology
platts_pure_gas['UTYPE_ORIGIN'] = platts_pure_gas['UTYPE']
platts_pure_gas['UTYPE'] = platts_pure_gas['UTYPE'].replace({'GT/C':'CCGT',
                                                             'ST/C':'CCGT',
                                                             'GT/CP':'CCGT',
                                                             'ST/CP':'CCGT',
                                                             'IC/H':'IC',
                                                             'IC/CD':'IC',
                                                             'CCSS':'CS',
                                                             'CCSS/P':'CS',
                                                             'GT/S':'GT',
                                                             'ST/S':'ST',
                                                             'GT/H':'GT',
                                                             'ST/D':'ST',
                                                             'GT/D':'GT'})
temp = platts_pure_gas.groupby(['LOCATIONID','UTYPE']).agg({'MW':'sum'})
platts_gas_utype_proportion = temp.groupby(level = 0).apply(lambda x:x/float(x.sum()))
platts_gas_utype_proportion.reset_index(inplace=True)
platts_id2utype_df = platts_gas_utype_proportion.loc[platts_gas_utype_proportion['MW'] >= 0.95,['LOCATIONID','UTYPE']]
platts_id2utype = platts_id2utype_df.set_index('LOCATIONID').to_dict()['UTYPE']

In [11]:
# impute unit type values for GPPD
platts_id2utype_df['LOCATIONID'] = platts_id2utype_df['LOCATIONID'].apply(str)
gppd_with_utype = pd.merge(gppd,platts_id2utype_df,'left',left_on='wepp_id',right_on='LOCATIONID')

### Run Estimator

In [33]:
output_col_prefix = 'ai4e_estimated_generation_'
baseline_prefix = 'baseline_generation_'
reported_prefix = 'generation_gwh_'

for i in range(len(gppd_with_utype)):
    if i % 100 == 0:
        print(i)
    
    # get fuel type from GPPD
    ft = gppd_with_utype.loc[i,'primary_fuel']
    
    # skip if the fuel type is not in ['Wind','Solar','Hydro','Gas']
    if ft not in model_params_map:
        continue
    
    # gas model estimates values raging from 2014 to 2018, other models can also estimate generation in 2013
    year_range = list(range(2014, 2018)) if ft == 'Gas' else list(range(2013, 2018))
    
    # fetch the parameter set for the particular fuel type
    params_set = model_params_map[ft]
    
    # select corresponding columns from the GPPD
    needed_gppd_cols = [param2column[p] for p in params_set]
    col_values = gppd_with_utype.loc[i, needed_gppd_cols]
    
    
    for year in year_range:
        
        # Skip this row if the generation is reported
        reported_generation_col = reported_prefix + str(year)
        if reported_generation_col in gppd_with_utype and not np.isnan(gppd_with_utype.loc[i,reported_generation_col]):
            continue
        
        # specify estimating year and put into the feature dict
        feature_values = {p:col_values[param2column[p]] for p in params_set}
        feature_values['estimating_year'] = year
        
        # Skip this row if the estimating year is earlier than the commissioning year
        if 'commissioning_year' in feature_values and year <= feature_values['commissioning_year']:
            continue
        
        # get number of days in the estimating year
        days = 366 if year % 4 == 0 else 365
        
        
        
        
        # create the baseline column name
        baseline_col = baseline_prefix + str(year)

        # skip this year if it's already estimated
        if baseline_col in gppd_with_utype.columns and not np.isnan(gppd_with_utype.loc[i,baseline_col]):
            continue
        
        # run baseline model
        try:
            cf = estimator.cf_getter.retrieve_capacity_factor(year, feature_values['country'], ft)
        except Exception as e:
            cf = np.NaN

        # calculate and clip baseline generation to zero if the result is subzero
        gen = cf * feature_values['capacity_mw'] * days * 24 / 1000
        if gen < 0:
            gen = 0
        gppd_with_utype.loc[i, baseline_col] = gen
        
        
        
        
        # create the ai4e column name
        output_col = output_col_prefix + str(year)
        
        # skip this year if it's already estimated
        if output_col in gppd_with_utype.columns and not np.isnan(gppd_with_utype.loc[i,output_col]):
            continue
        # run ai4e model
        try:
            cf, model_name = estimator.estimates(**feature_values)
        except Exception as e:
            cf, model_name = np.NaN, np.NaN
        
        # calculate and clip ai4e generation to zero if the result is subzero
        gen = cf * feature_values['capacity_mw'] * days * 24 / 1000
        if gen < 0:
            gen = 0
        gppd_with_utype.loc[i, output_col] = gen
        gppd_with_utype.loc[i, 'ai4e_model'] = model_name
    

0
af
ar
as
au
eu
gr
na
sa
si
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000

In [113]:
# Make annotations for missing generation

gppd_new = gppd_with_utype

missing_generation_reasons = {0:'no model for the fuel type', 
                              1:'missing commissioning year', 
                              2:'missing unit type',
                              3:'estimating year earlier than year of commissioning',
                              4:'actual generation reported',
                              5:'no gas estimation for 2013',
                              6:'other'}

year_range = ['2013', '2014', '2015', '2016', '2017']

def cell_is_null(df, index, col):
    return df.loc[index, col] != df.loc[index, col]

for i in range(gppd_new.shape[0]):
    fuel_type = gppd_new.loc[i,'primary_fuel']
    
    for year in year_range:
        
        col = 'no_estimation_annotation_' + year
        
        if fuel_type not in model_params_map:
            gppd_new.loc[i,col] = missing_generation_reasons[0]

        if not cell_is_null(gppd_new, i, 'generation_gwh_' + year):
            gppd_new.loc[i,col] = missing_generation_reasons[4]
            continue

        if fuel_type == 'Gas':
            if year == '2013':
                gppd_new.loc[i,col] = missing_generation_reasons[5]
            elif not cell_is_null(gppd_new, i, 'ai4e_estimated_generation_' + year):
                continue
            elif cell_is_null(gppd_new,i,'commissioning_year'):
                gppd_new.loc[i,col] = missing_generation_reasons[1]
            elif cell_is_null(gppd_new,i,'UTYPE'):
                gppd_new.loc[i,col] = missing_generation_reasons[2]
            elif gppd_new.loc[i, 'commissioning_year'] >= int(year):
                gppd_new.loc[i,col] = missing_generation_reasons[3]
            else:
                gppd_new.loc[i,col] = missing_generation_reasons[6]

        if fuel_type in ['Solar','Wind','Hydro']:
            if not cell_is_null(gppd_new, i, 'ai4e_estimated_generation_' + year):
                continue
            elif cell_is_null(gppd_new,i,'commissioning_year'):
                gppd_new.loc[i,col] = missing_generation_reasons[1]
            elif gppd_new.loc[i, 'commissioning_year'] >= int(year):
                gppd_new.loc[i,col] = missing_generation_reasons[3]
            else:
                gppd_new.loc[i,col] = missing_generation_reasons[6]

    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [146]:
# Drop the location id and unit type columns
gppd_new.drop(['LOCATIONID','UTYPE'], axis = 1, inplace = True)

In [147]:
# Save the updated database to csv
gppd_new.to_csv(OUTPUT_PATH, index=False)